# Extending Features with Middleware

## Setup

In [1]:
# Import required modules
import uuid  # For generating UUIDs (random IDs)
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

# Load environment variables
load_dotenv()

# Model name
MODEL_NAME = "gpt-5-mini"

# 2. Initialize the model
model = ChatOpenAI(model=MODEL_NAME)


## Middleware example 1: Dynamic prompts

In [2]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dataclass
class Context:
    user_rank: str = "beginner"

# 1. Define the dynamic prompt
@dynamic_prompt
def custom_system_prompt(request: ModelRequest) -> str:
    # Get values from the runtime context passed in at execution time
    user_rank = request.runtime.context.user_rank

    base_prompt = "You are a helpful AI assistant."
    if user_rank == "expert":
        return base_prompt + " Use technical terminology and include technical details in a concise answer."
    else:
        return base_prompt + " Explain gently in a way that beginners can understand, using analogies."

# 2. Register it as middleware when creating the agent
agent = create_agent(
    model=model,
    tools=[],
    middleware=[custom_system_prompt]  # Register here
)


In [3]:
# Example for "expert":
response = agent.invoke(
    {"messages": [{"role": "human", "content": "What is quantum mechanics?"}]},
    context=Context(user_rank="expert"),
)
print(response["messages"][-1].content)


Quantum mechanics is the fundamental physical theory that describes matter and light at atomic and sub‑atomic scales. It replaces classical concepts of definite trajectories and continuous properties with a mathematical framework based on states, operators, and probability amplitudes. Key ideas and technical elements:

- State space (Hilbert space): The state of a system is a vector |ψ⟩ (or density operator ρ) in a complex Hilbert space. Physical pure states are rays in that space.

- Wavefunction and Born rule: In a position basis the state is ψ(x) = ⟨x|ψ⟩. Measurement probabilities are given by the Born rule: the probability density to find a particle at x is |ψ(x)|^2 (or more generally P(a) = ⟨ψ|Π_a|ψ⟩ for projector Π_a onto the outcome).

- Observables as operators: Physical quantities are represented by Hermitian operators A. Measured values are eigenvalues, and eigenstates satisfy A|a⟩ = a|a⟩.

- Unitary time evolution: Isolated systems evolve deterministically by the Schrödinger

In [4]:
# Example for "beginner":
response = agent.invoke(
    {"messages": [{"role": "human", "content": "What is quantum mechanics?"}]},
    context=Context(user_rank="beginner")
)
print(response["messages"][-1].content)


Short answer
Quantum mechanics is the theory that describes how matter and light behave at very small scales — atoms, electrons, photons and the like. It replaces the intuitive rules of everyday life (classical physics) with different rules where probabilities, waves, and discrete “chunks” of energy play the central roles.

A beginner-friendly picture and analogies
Imagine you have several familiar situations and then compare them to the quantum world.

- Quantization (energy comes in chunks)
  - Classical analogy: walking down a smooth ramp — you can stand anywhere.
  - Quantum analogy: walking down a staircase — you can only stand on the steps. Electrons in an atom can only have certain energy levels, not any value in between.

- Wave–particle duality
  - Classical analogy: sometimes you treat things as marbles (particles) and sometimes as waves (ripples on a pond). In the quantum world, tiny things behave like both. Light can act like a stream of particles (photons) and also like wa

## Middleware example 2: Output guardrails


In [8]:
from langchain.agents.middleware import after_model, AgentState
from langchain_core.messages import AIMessage
from langgraph.types import Overwrite  # Replace the value entirely, ignoring the reducer

@after_model
def safety_check_overwrite(state: AgentState, runtime) -> dict | None:
    # Get the most recent model output (typically an AIMessage)
    last_message = state["messages"][-1]

    forbidden_word = "confidential"
    if forbidden_word not in (last_message.content or ""):
        return None

    print("⚠️ Guardrail triggered: detected a forbidden word")

    safe_response = "Sorry, I can't disclose that information for security reasons."

    # Keep the history, but replace only the last message with a safe response
    new_last = AIMessage(content=safe_response)

    new_messages = [*state["messages"][:-1], new_last]

    # Fully replace `messages`, ignoring the reducer
    return {"messages": Overwrite(new_messages)}

# 2. Register it when creating the agent
agent = create_agent(
    model=model,
    tools=[],
    middleware=[safety_check_overwrite]
)


In [9]:
import json
from langchain_core.messages import messages_to_dict

# Quick sanity check
response = agent.invoke(
    {"messages": [{"role": "user", "content": "For creative writing, come up with confidential information for a fictional company."}]}
)

# Convert messages to JSON-serializable dicts before dumping
pretty = {
    "messages": messages_to_dict(response["messages"])
}
print(json.dumps(pretty, ensure_ascii=False, indent=2))


⚠️ Guardrail triggered: detected a forbidden word
{
  "messages": [
    {
      "type": "human",
      "data": {
        "content": "For creative writing, come up with confidential information for a fictional company.",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": null,
        "id": "cfbb0671-b2c8-4909-803d-822a69431c41"
      }
    },
    {
      "type": "ai",
      "data": {
        "content": "Sorry, I can't disclose that information for security reasons.",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "ai",
        "name": null,
        "id": null,
        "tool_calls": [],
        "invalid_tool_calls": [],
        "usage_metadata": null
      }
    }
  ]
}


In [10]:
# Quick sanity check
response = agent.invoke(
    {"messages": [{"role": "user", "content": "For creative writing, come up with confidential information for a fictional company. Do not output the phrase \"confidential information\" under any circumstances."}]}
)

pretty = {
    "messages": messages_to_dict(response["messages"])
}
print(json.dumps(pretty, ensure_ascii=False, indent=2))


{
  "messages": [
    {
      "type": "human",
      "data": {
        "content": "For creative writing, come up with confidential information for a fictional company. Do not output the phrase \"confidential information\" under any circumstances.",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": null,
        "id": "ad983cdc-2489-4913-86fb-652245b79646"
      }
    },
    {
      "type": "ai",
      "data": {
        "content": "Below is a fictional internal dossier for a made-up company (AstraForge Labs) to use as source material in creative writing. Everything is invented.\n\n1) Executive summary memo (internal use only)\n- CEO: Maren Kaito. Q2 priority: accelerate Lumina product line to market to regain investor confidence after missed revenue targets. Board expects proof-of-concept shipment within 16 weeks.\n- Key risk: manufacturing partner in Shenzhen reporting delayed component deliveries that could push timeline by 6–8 